In [1]:
# Imports
import pandas as pd
import numpy as np

"""
Read Excel file. Combine sheets. Convert labels of relevant columns to yes/no. Save to CSV.
"""


'\nRead Excel file. Combine sheets. Convert labels of relevant columns to yes/no. Save to CSV.\n'

In [6]:
# Definitions
filename = r"../../data/data_full.xlsx"

sheet_names = ["Arto", "Evanfiya", "James", "Juho", "Sami"]

def load_spreadsheet(filepath,  add_index=False):
    sheets = pd.read_excel(filename, sheet_name=sheet_names, index_col=0 if add_index else False)
    return sheets

def label_data(df, slicer, predicate, label="label", use_all=True):
    reducer = all if use_all else any
    labels = (df.iloc[slicer] # Pick relevant data
        .apply(predicate) # Apply predicate
        .apply(reducer, axis=1) # Reduce to single boolean column
        .apply(lambda label : "yes" if label else "no")) # Convert back to yes/no
    labels.name = label
    return labels

In [7]:
sheets = load_spreadsheet(filename)
df = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title")

#Drop first column
df = df[df.columns[1:]]

df

,title,problemDescription,exampleSolution,starterCode,tests,theme,topic,concept,difficulty,id,The exercise description was clear (Yes/Partially/No),The exercise description matched the selected theme (Yes/Partially/No),The exercise description matched the selected topic (Yes/Partially/No),The exercise description matched the selected concept (Yes/No),Included concepts that were too advanced (Yes/No)\n,The exercise difficulty matched the selected difficulty (Too easy/Okay/Too difficult),Shallow vs deep personalization (Deep/Shallow/Unsure),Open field
0,Agatha Christie's Novel Ratings,"Agatha Christie, the famous novelist, has a ra...","{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'dart:async'; import 'pa...",literature,Agatha Christie,conditional statements,advanced,1.885152e+14,yes,yes,yes,yes,no,too easy,shallow,NaN
1,Painting Assessment,"In a painting class, the paintings are assesse...","{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'dart:async'; import 'pa...",handicrafts,painting,conditional statements,normal,1.974788e+14,yes,yes,yes,yes,no,okay,shallow,NaN
2,Lingonberry Sauce!,Write a program that asks the user for their f...,"{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'package:test/test.dart';...",food,Lingonberry sauce,user input,normal,-3.571840e+13,yes,yes,yes,yes,no,okay,shallow,NaN
3,Mozart's Symphony,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...","{'code': ""import 'dart:io';main() {}""}","{'testCode': ""import 'package:test/test.dart';...",classical music,Wolfgang Amadeus Mozart,program output,normal,5.950193e+14,yes,yes,yes,yes,no,okay,shallow,NaN
4,Calculate Monopoly rent,"In Monopoly, the rent a player must pay when t...","{'code': ""import 'dart:io'; main() { var ho...","{'code': ""import 'dart:io'; main() { } ""}","{'testCode': ""import 'dart:async'; import 'pa...",board games,Monopoly,arithmetics,normal,1.921704e+14,partially,yes,yes,yes,no,okay,deep,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Snowboarding Experience!,Write a program that asks the user for their f...,"{'code': ""import 'dart:io';main() { print('Wh...","{'code': ""import 'dart:io';main() {}""}","{'testCode': ""import 'package:test/test.dart';...",outdoor activities,snowboarding,user input,normal,-6.731519e+13,yes,yes,yes,yes,no,okay,unsure,NaN
46,Calculate Points for Carcassonne Game,"In the board game Carcassonne, players earn po...","{'code': ""import 'dart:io'; main() { var ci...","{'code': ""import 'dart:io'; main() { } ""}","{'testCode': ""import 'dart:async'; import 'pa...",board games,Carcassonne,variables,advanced,-4.134578e+13,yes,yes,yes,yes,no,okay,shallow,finally a proper advanced arithmetic exercise ...
47,Decorating the Christmas tree,Write a program that asks the user for their f...,"{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'package:test/test.dart';...",Christmas,decorating the tree,user input,normal,3.242888e+14,yes,yes,yes,yes,no,okay,shallow,NaN
48,Music Enthusiast,Write a program that asks the user for their f...,"{'code': ""import 'dart:io'; main() { print(...","{'code': ""import 'dart:io'; main() { }""}","{'testCode': ""import 'package:test/test.dart';...",classical music,Henry Purcell,user input,normal,5.498009e+14,yes,yes,yes,yes,no,okay,shallow,NaN


In [ ]:
slice_s = 11
slice_e = 14
eval_cols = df.columns[slice_s:slice_e]

# Pick cols 11-13 amd map labels yes => yes, no or partially => no
yes_df = df[eval_cols].eq("yes").map(lambda x: "yes" if x else "no")

#df = df.drop_duplicates("title")
df[eval_cols] = yes_df[eval_cols]

In [ ]:
slice_obj = np.s_[:, slice_s:slice_e]
pred = lambda s: s.str.lower() == "yes"
label = "All metrics are correct (Yes/No)"

result2 = pd.concat([df, label_data(df, slice_obj, pred, label)], axis=1)

result2.head()

In [ ]:
# Save label data

#label_cols_idx = [0, 11, 12, 13, 18]
#label_cols = result2.columns[label_cols_idx]
#only_labels = result2[label_cols]
#only_labels.to_csv("full_data_labels.csv", sep=";", index=False)

In [ ]:
stats = [
    *range(slice_s, slice_e),
    len(result2.columns) - 1
]

count_yes = (result2.iloc[:, stats] == "yes").sum()
count_no = len(result2) - count_yes
percentages = count_yes / len(result2)

df_stats = pd.DataFrame(data={"Correct (Yes)": count_yes, "Incorrect (Partially/No)": count_no, "Percentage of correct": percentages})

print(
    df_stats.to_latex()
    .replace("tabular}", "tabularx}{\\textwidth}", 1)
    .replace("tabular}", "tabularx}")
)


#df[label].where(lambda x : x == "yes").dropna().count() / len(df[label])

In [ ]:
#result2.to_csv("out.csv", sep=";", columns=["title", "topic", "theme", "concept", "problemDescription", "exampleSolution", *eval_cols, label], index=False)